<a href="https://colab.research.google.com/github/X-T-E-R/GPT-SoVITS-Inference/blob/stable/colab_webui.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

环境配置 environment

In [ ]:
%pip install -q condacolab
%pip install ipykernel

# Setting up condacolab and installing packages
import condacolab
condacolab.install_from_url("https://repo.anaconda.com/miniconda/Miniconda3-py39_23.11.0-2-Linux-x86_64.sh")
%cd -q /content
!git clone https://github.com/X-T-E-R/GPT-SoVITS-Inference
!conda install -y -q -c pytorch -c nvidia cudatoolkit
%cd -q /content/GPT-SoVITS-Inference
!conda install -y -q -c conda-forge gcc gxx ffmpeg cmake -c pytorch -c nvidia
!/usr/local/bin/pip install -r requirements.txt

In [ ]:
# @title Download pretrained models 下载预训练模型
!mkdir -p /content/GPT-SoVITS/GPT_SoVITS/pretrained_models

%cd /content/GPT-SoVITS-Inference/GPT_SoVITS/pretrained_models
!git clone https://huggingface.co/lj1995/GPT-SoVITS

!git config core.sparseCheckout true
!mv /content/GPT-SoVITS-Inference/GPT_SoVITS/pretrained_models/GPT-SoVITS/* /content/GPT-SoVITS-Inference/GPT_SoVITS/pretrained_models/

设置 settings

In [ ]:
# settings
%cd /content/GPT-SoVITS-Inference
import json
config = {}
try:
    with open("Inference/config.json", "r", encoding="utf-8") as f:
        config = json.load(f)
except FileNotFoundError:
    raise FileNotFoundError("config.json not found, please check the file path.")

config["locale"] = "en-US"
config["is_share"] = "true"
config["models_path"] = "trained"

try:
    with open("Inference/config.json", "w", encoding="utf-8") as f:
        json.dump(config, f, ensure_ascii=False, indent=4)
except FileNotFoundError:
    raise FileNotFoundError("config.json not found, please check the file path.")

In [11]:
%cd /content/GPT-SoVITS-Inference
#@title Import model 导入模型 (HuggingFace)

# put your character folder in trained/
# or use the code below to download the HuggingFace model
# please check if the zip contains the model files and a audio_file named by its content
# if not, please modify the folder manually in trained/ 

import requests
import zipfile
import shutil
import os
import urllib.parse


# special thanks to baicai1145 for providing the model(and its copyright belongs to the Mihoyo)
# copy the link from the download button on the model page
hf_link = 'https://huggingface.co/baicai1145/GPT-SoVITS-STAR/resolve/main/%E9%93%B6%E7%8B%BC.zip?download=true' #@param {type: "string"}

# get the name of the character folder, or you can set it manually
character_name = urllib.parse.unquote(os.path.basename(hf_link).rsplit('.', 1)[0])

print(f'character_name: {character_name}')

output_path = os.path.join('trained', character_name)
os.makedirs(output_path, exist_ok=True)

zip_file_path = os.path.join(output_path, 'file.zip')

# download the zip file
response = requests.get(hf_link)
with open(zip_file_path, 'wb') as file:
    file.write(response.content)

# extract the zip file
import zipfile

def get_decoder(file_name: str):
    try:
        return file_name.encode('cp437').decode('gbk')
    except:
        return file_name


with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    for file_info in zip_ref.infolist():
        # 解码并重新编码文件名
        encoded_file_name = get_decoder(file_info.filename)
        new_path = os.path.join(output_path, encoded_file_name)
        source = zip_ref.open(file_info.filename)
        # 判断f是否是目录，目录的结尾是'/'或'\'
        if encoded_file_name[-1] not in ['\\','/']:
            with open(new_path,'wb') as file:
                file.write(zip_ref.read(file_info.filename))
                file.close()
        else:
            os.makedirs(new_path, exist_ok=True)


Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Note: you may need to restart the kernel to use updated packages.
character_name: 银狼


NotRarFile: Not a RAR file

In [ ]:
# @title launch WebUI 启动WebUI
# special thanks to @rc4 
%pip install ipykernel
%cd /content/GPT-SoVITS-Inference/
!/usr/local/bin/python  app.py